После загрузки изображений с сайта Кинопоиск получены три папки: **sample_dataset**, в которой хранятся единичные качественные изображения знаменитостей, которые могут быть использованы в качестве эталонных; **gallery**, в которой могут находится до пяти дополнительных изображений; **images**, в которой также содержится некоторое количество дополнительных изображений. 

Дополнительные изображения разного качества. Для очень популярных знаменитостей их количество в папке **images** может достигать нескольких сотен. Поэтому, целесообразно отобрать из них наиболее релевантные изображения лица для каждой знаменитости.

Для этого создадим фнукцию, которая возьмёт список знаменитостей из папки **sample_dataset** и, сохрнаяя разеление знаменитостей на отечественных и иностранных, получит доступные дополнительные изображения для каждой знаменитости. Для каждого дополнительного изображения будет вычислен эмбединг на основе библиотеки *face_recognition* и сопоставлен с эмбедином эталонных изображений. По косинусному расстоянию между эмбедингом эталонного изображения и эмбедигами остальных изображений будет произведён отбор наиболее релевантных изображений.

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import notebook
import face_recognition
import pandas as pd
import numpy as np
import pickle
import shutil
import os, sys
import glob
import math

In [2]:
from sklearn.cluster import AgglomerativeClustering

In [3]:
from PIL import UnidentifiedImageError

In [4]:
# функция отбора изображений для обучающего датасета

def selection_images(gender='men'):
    pet_path = './Documents/DataScience/PET-project'
    
    # Создание папки под финальный датасет
    if os.path.exists(f'{pet_path}/train') != True:
        os.mkdir(f'{pet_path}/train')
    
    # Создание подпапок для актеров и актрис
    if os.path.exists(f'{pet_path}/train/{gender}') != True:
        os.mkdir(f'{pet_path}/train/{gender}')
    
    nation = ['eng', 'rus']
    for natio in nation:
        # Создание подпапок для разделения знаменитостей на иностранных и отечественных
        if os.path.exists(f'{pet_path}/train/{gender}/{natio}') != True:
            os.mkdir(f'{pet_path}/train/{gender}/{natio}')
        
        # Получаем список знаменитостей из папки с эталонными изображениями
        folders = glob.glob(f'{pet_path}/sample_dataset/{gender}/{natio}/*')
                            
        # Производим перебор списка знаменитостей с отбором наилучших изображений
        for folder in notebook.tqdm(folders):
            if natio == 'eng':
                name = folder[folder.find('eng\\') + 4:].strip()
            elif natio == 'rus':
                name = folder[folder.find('rus\\') + 4:].strip()
            
            if os.path.exists(f'{pet_path}/train/{gender}/{natio}/{name}') != True:
                os.mkdir(f'{pet_path}/train/{gender}/{natio}/{name}')
                print(f'Directory for {name} created!')
            else:
                print(f'Images for {name} already uploaded!')
                continue
            
            # Путь к эталонным изображениям
            path_to_sample = f'{pet_path}/sample_dataset'
            
            # Путь к галерее дополнтельных изображений знаменитостей
            path_to_gallery = f'{pet_path}/gallery'
            
            # Путь к расширенному набору изображений знаменитостей
            path_to_images = f'{pet_path}/images'
            
            # Формируем список файлов для последующего отбора
            sample_file = os.listdir(f'{path_to_sample}/{gender}/{natio}/{name}')
            files =  [f'{path_to_sample}/{gender}/{natio}/{name}/' + file for file in sample_file]
            
            try:
                gallery_files = os.listdir(f'{path_to_gallery}/{gender}/{natio}/{name}')
                [files.append(f'{path_to_gallery}/{gender}/{natio}/{name}/' + file) for file in gallery_files if \
                os.stat(f'{path_to_gallery}/{gender}/{natio}/{name}/' + file).st_size > 0]
                
                images_files = os.listdir(f'{path_to_images}/{gender}/{natio}/{name}')
                [files.append(f'{path_to_images}/{gender}/{natio}/{name}/' + file) for file in images_files if \
                os.stat(f'{path_to_images}/{gender}/{natio}/{name}/' + file).st_size > 0]
            except FileNotFoundError:
                try:
                    print(f'Path :: {path_to_gallery}/{gender}/{natio}/{name} - not found!')
                    shutil.copy(f'{path_to_sample}/{gender}/{natio}/{name}/256.jpg',\
                                f'{pet_path}/train/{gender}/{natio}/{name}/Image_0.jpg')
                except FileNotFoundError:
                    print(f'File :: {path_to_sample}/{gender}/{natio}/{name}/256.jpg - not found!')
                    continue
                continue
            
            # Вычисляем эмбединги для лиц на каждом изображении
            embedings = []
            for file in files:
                try:
                    sample_face = face_recognition.load_image_file(file)
                    embedings.append(face_recognition.face_encodings(sample_face)[0])
                except (IndexError, UnidentifiedImageError):
                    print(f'Not found face in {file}')
                    continue
            
            if len(embedings) > 1:
                # Группировка изображений по кластерам для последующего отбора
                agg = AgglomerativeClustering(n_clusters=None, distance_threshold=0.1,\
                                              affinity='cosine', linkage='complete').fit(embedings)
                
                # Сохраняем в переменных полученные лейблы и количество классов
                labels = agg.labels_
                n_classes = len(set(labels))
                
                # Разбираем файлы по полученным группам
                files_by_group = [[] for x in range(n_classes)]
                
                for x in range(n_classes):
                    for j in range(len(labels)):
                        if labels[j] == x:
                            files_by_group[x].append(files[j])
                            
                # Определяем самый большой кластер
                max_labels_count = 0
                
                for x in range(1, n_classes):
                    if len(files_by_group[x]) > len(files_by_group[max_labels_count]):
                        max_labels_count = x
                        
                # Находим в какой кластер попало эталонное изображение
                group_with_sample = 0
                
                for x in range(n_classes):
                    if f'{path_to_sample}/{gender}/{natio}/{name}/256.jpg' in files_by_group[x]:
                        group_with_sample = x
                        break
                        
                # Формируем словарь с ключами в виде имени файла и значениями в виде эмбедингов
                dictionary = dict(zip(files, embedings))
                
                # Проверяем совпадает ли номер наибольшего кластера с номером кластера, в котором обнаружено эталонное 
                # изображение. Если нет - то объединяем эти два кластера.
                if max_labels_count == group_with_sample:
                    main_files = files_by_group[max_labels_count]
                else:
                    main_files = files_by_group[group_with_sample] + files_by_group[max_labels_count]
                    
                # Вычисляем косинусное расстояние, сохраняем его в словарь и по нему отбираем наилучшие файлы как наиболее
                # близкие к эталонному изображению
                rates = dict()
                
                for f in main_files[1:]:
                    sample_key = f'{path_to_sample}/{gender}/{natio}/{name}/256.jpg'
                    cos = cosine_similarity(dictionary[sample_key].reshape(1, -1), dictionary[f].reshape(1, -1))[0][0]
                    rates[cos] = f
                    
                sorted_rates = list(rates.keys())
                sorted_rates.sort(reverse=True)
                
                best_images = [rates[x] for x in sorted_rates if 0.99 > x > 0.93]
                
                # Рекурсивная функция для удаления дубликатов из отобранных изображений
                def doubles_cleaner(images, pos, dictionary=dictionary):
                    if pos == len(images)-1:
                        return(images)
                    else:
                        for x in range(pos+1, len(images)):
                            if cosine_similarity(dictionary[images[pos]].reshape(1, -1),\
                                                 dictionary[images[x]].reshape(1, -1))[0] > 0.98:
                                del images[x]
                                return doubles_cleaner(images, pos=pos+1)
                            return doubles_cleaner(images, pos=pos+1)
                        
                try:
                    doubles_cleaner(best_images, pos=0)
                except:
                    pass
                
                # Добавляем в начало списка отобранных файлов эталонное изображение
                best_images.insert(0, sample_key)
                
                # сохранение полученных изображений в итоговый тренировочный датасет
                i = 0
                for best_image in best_images:
                    shutil.copy(best_image, f'{pet_path}/train/{gender}/{natio}/{name}/Image_{i}.jpg')
                    i += 1
            else:
                try:
                    shutil.copy(f'{path_to_sample}/{gender}/{natio}/{name}/256.jpg',\
                                f'{pet_path}/train/{gender}/{natio}/{name}/Image_0.jpg')
                except FileNotFoundError:
                    print(f'File :: {path_to_sample}/{gender}/{natio}/{name}/256.jpg - not found!')
                    continue

In [7]:
selection_images(gender='women')

  0%|          | 0/2630 [00:00<?, ?it/s]

Images for A.J. Cook already uploaded!
Images for Aaliyah already uploaded!
Images for Aarti Mann already uploaded!
Images for Abbey Lee already uploaded!
Images for Abbie Cornish already uploaded!
Images for Abby Simpson already uploaded!
Images for Abella Danger already uploaded!
Images for Abigail Breslin already uploaded!
Images for Abigail Cowen already uploaded!
Images for Abigail Spencer already uploaded!
Images for Ada Nicodemou already uploaded!
Images for Addison Rae already uploaded!
Images for Addison Timlin already uploaded!
Images for Adelaide Kane already uploaded!
Images for Adele already uploaded!
Images for Adina Porter already uploaded!
Images for Adjoa Andoh already uploaded!
Images for Adria Arjona already uploaded!
Images for Adriana Barraza already uploaded!
Images for Adriana Chechik already uploaded!
Images for Adriana Lima already uploaded!
Images for Adriana Ugarte already uploaded!
Images for Adrianne Palicki already uploaded!
Images for Adrienne Shelly alre

Images for Holly Earl already uploaded!
Images for Holly Marie Combs already uploaded!
Images for Honey Wilder already uploaded!
Images for Hope Davis already uploaded!
Images for Hudson Leick already uploaded!
Images for Hunter Schafer already uploaded!
Images for Hwang Jeong-eum already uploaded!
Images for Hyeri already uploaded!
Images for Hélène Rollès already uploaded!
Images for Iben Dorner Østergaard already uploaded!
Images for Ida Elise Broch already uploaded!
Images for Ida Engvoll already uploaded!
Images for Idina Menzel already uploaded!
Images for Ilfenesh Hadera already uploaded!
Images for Imane 'Pokimane' Anys already uploaded!
Images for Imani Hakim already uploaded!
Images for Imelda Staunton already uploaded!
Images for Imogen Daines already uploaded!
Images for Imogen Poots already uploaded!
Images for Imogen Waterhouse already uploaded!
Images for Inbar Lavi already uploaded!
Images for India Eisley already uploaded!
Images for India Mullen already uploaded!
Imag

Not found face in ./Documents/DataScience/PET-project/images/women/eng/Jessica Brown-Findlay/Image_41.jpg
Directory for Jessica Capshaw created!
Directory for Jessica Cauffiel created!
Directory for Jessica Chastain created!
Not found face in ./Documents/DataScience/PET-project/images/women/eng/Jessica Chastain/Image_55.jpg
Directory for Jessica Collins created!
Directory for Jessica De Gouw created!
Directory for Jessica Green created!
Not found face in ./Documents/DataScience/PET-project/images/women/eng/Jessica Green/Image_2.jpg
Not found face in ./Documents/DataScience/PET-project/images/women/eng/Jessica Green/Image_37.jpg
Directory for Jessica Hecht created!
Directory for Jessica Henwick created!
Directory for Jessica Lange created!
Directory for Jessica Lowndes created!
Not found face in ./Documents/DataScience/PET-project/images/women/eng/Jessica Lowndes/Image_16.jpg
Directory for Jessica Lucas created!
Directory for Jessica Madsen created!
Directory for Jessica McLeod created!

Not found face in ./Documents/DataScience/PET-project/gallery/women/eng/Julie Estelle/Image_3.jpg
Directory for Julie Gonzalo created!
Not found face in ./Documents/DataScience/PET-project/images/women/eng/Julie Gonzalo/Image_5.jpg
Directory for Julie Hagerty created!
Directory for Julie Walters created!
Not found face in ./Documents/DataScience/PET-project/images/women/eng/Julie Walters/Image_2.jpg
Directory for Juliet Stevenson created!
Not found face in ./Documents/DataScience/PET-project/gallery/women/eng/Juliet Stevenson/Image_4.jpg
Not found face in ./Documents/DataScience/PET-project/images/women/eng/Juliet Stevenson/Image_5.jpg
Directory for Julietta Stepanyan created!
Path :: ./Documents/DataScience/PET-project/gallery/women/eng/Julietta Stepanyan - not found!
Directory for Juliette Binoche created!
Not found face in ./Documents/DataScience/PET-project/images/women/eng/Juliette Binoche/Image_46.jpg
Not found face in ./Documents/DataScience/PET-project/images/women/eng/Juliette

...

Not found face in ./Documents/DataScience/PET-project/images/women/rus/Мария Фомина/Image_45.jpg
Directory for Мария Фортунатова created!
Directory for Мария Хижняк created!
Not found face in ./Documents/DataScience/PET-project/images/women/rus/Мария Хижняк/Image_18.jpg
Directory for Мария Цветкова-Овсянникова created!
Directory for Мария Чернолихова created!
Directory for Мария Шалаева created!
Directory for Мария Шукшина created!
Directory for Мария Шумакова created!
Directory for Марта Кесслер created!
Directory for Марта Носова created!
Directory for Маруся Зыкова created!
Directory for Маруся Климова created!
Directory for Марьяна Спивак created!
Not found face in ./Documents/DataScience/PET-project/images/women/rus/Марьяна Спивак/Image_6.jpg
Directory for Маша Лобанова created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Маша Лобанова - not found!
Directory for Маша Оамер created!
Directory for Мика Ардова created!
Path :: ./Documents/DataScience/PET-project/gal

Directory for Нина Зоткина created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Нина Зоткина - not found!
Directory for Нина Крачковская created!
Directory for Нина Маслова created!
Directory for Нина Русланова created!
Directory for Нина Тер-Осипян created!
Directory for Нина Ургант created!
Directory for Нина Усатова created!
Directory for Нинель Мышкова created!
Directory for Нино Нинидзе created!
Not found face in ./Documents/DataScience/PET-project/gallery/women/rus/Нино Нинидзе/Image_2.jpg
Not found face in ./Documents/DataScience/PET-project/images/women/rus/Нино Нинидзе/Image_1.jpg
Not found face in ./Documents/DataScience/PET-project/images/women/rus/Нино Нинидзе/Image_23.jpg
Directory for Нонна Гришаева created!
Directory for Нонна Мордюкова created!
Directory for Нонна Тен created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Нонна Тен - not found!
Directory for Нюша Шурочкина created!
Not found face in ./Documents/DataScience/PET-project/i

Directory for Пирет Мянгел created!
Directory for Полина Агуреева created!
Directory for Полина Ауг created!
Directory for Полина Виторган created!
Directory for Полина Войченко created!
Directory for Полина Гагарина created!
Directory for Полина Гухман created!
Directory for Полина Долиндо created!
Directory for Полина Кузьминская created!
Directory for Полина Кутепова created!
Directory for Полина Куценко created!
Directory for Полина Максимова created!
Directory for Полина Носыхина created!
Directory for Полина Плучек created!
Directory for Полина Пушкарук created!
Directory for Полина Степанова created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Полина Степанова - not found!
Directory for Полина Сыркина created!
Directory for Полина Федина created!
Not found face in ./Documents/DataScience/PET-project/gallery/women/rus/Полина Федина/Image_2.jpg
Not found face in ./Documents/DataScience/PET-project/gallery/women/rus/Полина Федина/Image_3.jpg
Not found face in ./Do

Directory for Тамара Обутова created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Тамара Обутова - not found!
Directory for Тамара Сёмина created!
Directory for Тамара Турава created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Тамара Турава - not found!
Directory for Таня Ю created!
Directory for Татьяна Абрамова created!
Directory for Татьяна Аксюта created!
Directory for Татьяна Арнтгольц created!
Directory for Татьяна Аугшкап created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Татьяна Аугшкап - not found!
Directory for Татьяна Бабенкова created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Татьяна Бабенкова - not found!
Directory for Татьяна Бовкалова created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Татьяна Бовкалова - not found!
Directory for Татьяна Бондаренко created!
Directory for Татьяна Бунина created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Татьяна Бунина - not

Directory for Яна Смоленцева created!
Directory for Яна Троянова created!
Not found face in ./Documents/DataScience/PET-project/images/women/rus/Яна Троянова/Image_11.jpg
Directory for Яна Шивкова created!
Directory for Янина Бугрова created!
Directory for Янина Жеймо created!
Directory for Янина Колесниченко created!
Path :: ./Documents/DataScience/PET-project/gallery/women/rus/Янина Колесниченко - not found!
Directory for Янина Лисовская created!
Directory for Янина Соколовская created!
Directory for Янина Студилина created!
Not found face in ./Documents/DataScience/PET-project/images/women/rus/Янина Студилина/Image_11.jpg
Directory for Ясмина Омерович created!


In [11]:
selection_images()

  0%|          | 0/2842 [00:00<?, ?it/s]

Images for 50 Cent already uploaded!
Images for A Martinez already uploaded!
Images for Aamir Khan already uploaded!
Images for Aaron Ashmore already uploaded!
Images for Aaron Eckhart already uploaded!
Images for Aaron Heffernan already uploaded!
Images for Aaron Murphy already uploaded!
Images for Aaron Paul already uploaded!
Images for Aaron Pierre already uploaded!
Images for Aaron Taylor-Johnson already uploaded!
Images for Aaron W Reed already uploaded!
Images for Aasif Mandvi already uploaded!
Images for Abhishek Bachchan already uploaded!
Images for Adam Arkin already uploaded!
Images for Adam Baldwin already uploaded!
Images for Adam Beach already uploaded!
Images for Adam Brody already uploaded!
Images for Adam Demos already uploaded!
Images for Adam Devine already uploaded!
Images for Adam DiMarco already uploaded!
Images for Adam Driver already uploaded!
Images for Adam Godley already uploaded!
Images for Adam Goldberg already uploaded!
Images for Adam Levine already upload

Images for Halit Ergenç already uploaded!
Images for Hamish Linklater already uploaded!
Images for Hank Azaria already uploaded!
Images for Hannibal Buress already uploaded!
Images for Hanno Koffler already uploaded!
Images for Hans Matheson already uploaded!
Images for Harold Perrineau already uploaded!
Images for Harris Dickinson already uploaded!
Images for Harrison Ford already uploaded!
Images for Harrison Gilbertson already uploaded!
Images for Harrison Osterfield already uploaded!
Images for Harry Connick Jr already uploaded!
Images for Harry Dean Stanton already uploaded!
Images for Harry Holland already uploaded!
Images for Harry J. Lennix already uploaded!
Images for Harry Lloyd already uploaded!
Images for Harry Melling already uploaded!
Images for Harry Shum Jr already uploaded!
Images for Harry Styles already uploaded!
Images for Harry Treadaway already uploaded!
Images for Harshvardhan Rane already uploaded!
Images for Hart Denton already uploaded!
Images for Harvey Guill

Images for Paul Walter Hauser already uploaded!
Images for Paul Wesley already uploaded!
Images for Paulo Costanzo already uploaded!
Images for Pauly Shore already uploaded!
Images for Pavel Lychnikoff already uploaded!
Images for Pavel Trávníček already uploaded!
Images for Pedro Alonso already uploaded!
Images for Pedro Pascal already uploaded!
Images for Penn Badgley already uploaded!
Images for Pete Davidson already uploaded!
Images for Pete Postlethwaite already uploaded!
Images for Peter Berg already uploaded!
Images for Peter Capaldi already uploaded!
Images for Peter Cushing already uploaded!
Images for Peter Dinklage already uploaded!
Images for Peter Facinelli already uploaded!
Images for Peter Falk already uploaded!
Images for Peter Fonda already uploaded!
Images for Peter Franzén already uploaded!
Images for Peter Friedman already uploaded!
Images for Peter Gadiot already uploaded!
Images for Peter Gallagher already uploaded!
Images for Peter Greene already uploaded!
Images

Not found face in ./Documents/DataScience/PET-project/images/men/eng/Tom Holland/Image_18.jpg
Not found face in ./Documents/DataScience/PET-project/images/men/eng/Tom Holland/Image_3.jpg
Directory for Tom Hollander created!
Directory for Tom Hopper created!
Directory for Tom Hulce created!
Directory for Tom Leeb created!
Directory for Tom Lister Jr created!
Directory for Tom McGrath created!
Directory for Tom Noonan created!
Directory for Tom Payne created!
Directory for Tom Prior created!
Directory for Tom Riley created!
Directory for Tom Schilling created!
Directory for Tom Selleck created!
Not found face in ./Documents/DataScience/PET-project/images/men/eng/Tom Selleck/Image_2.jpg
Directory for Tom Sizemore created!
Directory for Tom Sturridge created!
Not found face in ./Documents/DataScience/PET-project/gallery/men/eng/Tom Sturridge/Image_1.jpg
Directory for Tom Waits created!
Directory for Tom Welling created!
Not found face in ./Documents/DataScience/PET-project/images/men/eng/T

...

Directory for Михаил Дадыко created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Михаил Дадыко - not found!
Directory for Михаил Данилюк created!
Directory for Михаил Евдокимов created!
Directory for Михаил Евланов created!
Directory for Михаил Елисеев created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Михаил Елисеев - not found!
Directory for Михаил Ефремов created!
Directory for Михаил Жарковский created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Михаил Жарковский - not found!
Directory for Михаил Жаров created!
Directory for Михаил Жигалов created!
Directory for Михаил Казаков created!
Directory for Михаил Козаков created!
Directory for Михаил Кокшенов created!
Directory for Михаил Кононов created!
Directory for Михаил Кремер created!
Directory for Михаил Кузнецов created!
Directory for Михаил Кшиштовский created!
Directory for Михаил Мартьянов created!
Directory for Михаил Мишин created!
Path :: ./Documents/DataScience/PET-project

Directory for Олег Масленников created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Олег Масленников - not found!
Directory for Олег Масленников-Войтов created!
Directory for Олег Меньшиков created!
Directory for Олег Назаров created!
Directory for Олег Отс created!
Directory for Олег Протасов created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Олег Протасов - not found!
Directory for Олег Савосин created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Олег Савосин - not found!
Directory for Олег Стриженов created!
Directory for Олег Табаков created!
Directory for Олег Тактаров created!
Directory for Олег Фёдоров created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Олег Фёдоров - not found!
Directory for Олег Филипчик created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Олег Филипчик - not found!
Directory for Олег Фомин created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Олег Фомин - not fou

Directory for Святослав Тарасов created!
Directory for Семён Серзин created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Семён Серзин - not found!
Directory for Семён Стругачев created!
Directory for Семён Трескунов created!
Directory for Семён Шкаликов created!
Not found face in ./Documents/DataScience/PET-project/gallery/men/rus/Семён Шкаликов/Image_5.jpg
Directory for Семён Штейнберг created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Семён Штейнберг - not found!
Directory for Семен Соколовский created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Семен Соколовский - not found!
Directory for Семен Фарада created!
Directory for Сергей Апрельский created!
Directory for Сергей Арцибашев created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Сергей Арцибашев - not found!
Directory for Сергей Астахов created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Сергей Астахов - not found!
Directory for Сергей Бадюк crea

Directory for Сергей Чонишвили created!
Directory for Сергей Шакуров created!
Directory for Сергей Шарифуллин created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Сергей Шарифуллин - not found!
Directory for Сергей Шароватов created!
Directory for Сергей Швыдкой created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Сергей Швыдкой - not found!
Directory for Сергей Шнуров created!
Directory for Сергей Шнырев created!
Directory for Сергей Юрский created!
Directory for Сергей Юшкевич created!
Directory for Сергей Яценюк created!
Directory for Серджиу Финити created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Серджиу Финити - not found!
Directory for Сослан Фидаров created!
Directory for Спартак Мишулин created!
Directory for Спартак Сумченко created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Спартак Сумченко - not found!
Directory for Станислав Беляев created!
Directory for Станислав Боклан created!
Directory for Станислав Б

Directory for Юрий Саранцев created!
Directory for Юрий Сафаров created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Юрий Сафаров - not found!
Directory for Юрий Соломин created!
Directory for Юрий Степанов created!
Directory for Юрий Стоянов created!
Directory for Юрий Тарасов created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Юрий Тарасов - not found!
Directory for Юрий Торсуев created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Юрий Торсуев - not found!
Directory for Юрий Цурило created!
Directory for Юрий Чернов created!
Directory for Юрий Чурсин created!
Directory for Юрий Шерстнев created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Юрий Шерстнев - not found!
Directory for Юрий Шумило created!
Path :: ./Documents/DataScience/PET-project/gallery/men/rus/Юрий Шумило - not found!
Directory for Юрий Яковлев created!
Directory for Яков Левда created!
Directory for Яков Шамшин created!
Path :: ./Documents/DataScience/PE

Изображения отобраны. Визуальный анализ сохранённых изображений показал, что разброс количества фотографий лиц для отдельных знаменитостей все ещё велик и находится в диапазоне от ноля до полутора сотен. Поэтому, чтобы избежать такого дисбаланса, произведём следующее:
* удалим всех знаменитостей, у которых число изображений менее двух;
* ограничим максимальное число изображений 25 фотографими лиц для каждой знаменитости, удалив лишние в конце списка.

Напишем для этого соотвествующую функцию.

In [9]:
# функция для удаления лишних изображений

def deleting_excess_images(gender='men'):
    train_path = './Documents/DataScience/PET-project'
    
    nation = ['eng', 'rus']
    for natio in nation:
        # Получаем список знаменитостей
        folders = glob.glob(f'{train_path}/train/{gender}/{natio}/*')
        
        # Производим перебор списка знаменитостей
        for folder in notebook.tqdm(folders):
            if natio == 'eng':
                name = folder[folder.find('eng\\') + 4:].strip()
            elif natio == 'rus':
                name = folder[folder.find('rus\\') + 4:].strip()
                
            # Получаем список изображений для каждой знаменитости
            try:
                files = glob.glob(f'{train_path}/train/{gender}/{natio}/{name}/*')
                
                # Оставим только папки с количеством изображений 2 и более, а также если файлов больше 25 в папке, 
                # о оставляем только 25, первых, а остальные удаляем
                if len(files) < 2:
                    shutil.rmtree(f'{train_path}/train/{gender}/{natio}/{name}/', ignore_errors=True)
                    print(f'Directory :: {train_path}/train/{gender}/{natio}/{name}/ contains less that 2 images and deleted!')
                elif len(files) > 25:
                    # Получаем уникальные номера изображений в папках знаменитостей
                    file_numbers = []
                    for file in files:
                        f_name = file.split('\\')[-1]
                        file_numbers.append(int(f_name.replace('Image_', '').replace('.jpg', '')))
                        
                    # Сортировка списка
                    file_numbers.sort()
                    
                    # Получаем идентификаторы файлов для удаления
                    del_files = file_numbers[25:]

                    # Удаляем файлы
                    for num in del_files:
                         os.remove(f'{train_path}/train/{gender}/{natio}/{name}/Image_{str(num)}.jpg')
                    print(f'Excess images in folder :: {train_path}/train/{gender}/{natio}/{name} - successful deleted!')
            except FileNotFoundError:
                print(f'Path :: {train_path}/train/{gender}/{natio}/{name} - not found!')
                continue

In [10]:
deleting_excess_images()

  0%|          | 0/2842 [00:00<?, ?it/s]

Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/50 Cent - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Aaron Ashmore/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Aaron Eckhart - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Aaron Heffernan/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Aaron Murphy/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Aaron Paul - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Aaron Taylor-Johnson - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Aaron W Reed/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/e

Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Bruce Willis - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Bruno Ganz/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Bugzy Malone/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Burgess Meredith/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Calahan Skogman/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Caleb Landry Jones - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Callan McAuliffe - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Cameron Monaghan - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/en

Directory :: ./Documents/DataScience/PET-project/train/men/eng/David Burtka/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/David Costabile/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/David Duchovny - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/David Hayter/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/David Hemmings/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/David Jason/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/David Morrissey - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/David Oyelowo - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/D

Directory :: ./Documents/DataScience/PET-project/train/men/eng/Gordon Michael Woolvett/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Greg Wise/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Gregg Sulkin - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Gregory Peck - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Grégory Montel/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Guillaume Canet - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/eng/Guy Pearce - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Guðjón Pedersen/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/eng/Gök

...

Directory :: ./Documents/DataScience/PET-project/train/men/rus/OG Buda/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Августин Милованов/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Адель Аль-Хадад/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Азамат Нигманов/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Александр Адабашьян/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Александр Барановский/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Александр Бениаминов/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Александр Блок/ contains less that 2 images and deleted!
Directory :: ./Document

Directory :: ./Documents/DataScience/PET-project/train/men/rus/Артем Бордовский/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Артем Веселов/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Артем Калайджян/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Артем Карокозян/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Артем Кошман/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Артем Шевченко/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Арчил Гомиашвили/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Барасби Мулаев/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/

Directory :: ./Documents/DataScience/PET-project/train/men/rus/Даниил Пикула/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Данила Иванов/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/rus/Данила Козловский - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Данила Перов/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/rus/Данила Якушев - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/rus/Денис Васильев - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Денис Дорохов/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Денис Некрасов/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/ru

Directory :: ./Documents/DataScience/PET-project/train/men/rus/Максим Самчик/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Мамука Кикалейшвили/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Марат Амираев/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Марк Горонок/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/men/rus/Матвей Зубалевич - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Матвей Семёнов/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Микаэл Джанибекян/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Михаил Асанкин/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET

Directory :: ./Documents/DataScience/PET-project/train/men/rus/Сергей Селин/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Сергей Сосновский/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Сергей Степанченко/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Сергей Тихонов/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Сергей Угрюмов/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Сергей Уманов/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Сергей Черданцев/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/men/rus/Сергей Шарифуллин/ contains less that 2 images and deleted!
Directory :: ./Documents/DataSc

In [11]:
deleting_excess_images(gender='women')

  0%|          | 0/2630 [00:00<?, ?it/s]

Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/A.J. Cook - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Aaliyah - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Abbey Lee - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Abbie Cornish - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/eng/Abby Simpson/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Abella Danger - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Abigail Breslin - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Abigail Cowen - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Abigail 

Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Amy Adams - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Amy Beth Schumer - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/eng/Amy Huck/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Amy Poehler - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Amy Smart - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/eng/Amy Spanger/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Amybeth McNulty - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Ana de Armas - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/An

Directory :: ./Documents/DataScience/PET-project/train/women/eng/Barbora Kodetová/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Beanie Feldstein - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Becki Newton - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Bel Powley - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Bella Heathcote - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/eng/Bella Ramsey/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Bella Thorne - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/eng/Belén Cuesta/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-projec

Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Catherine Bell - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Catherine Tate - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Catherine Zeta-Jones - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/eng/Chandra Wilson/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/eng/Charissa Barton/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Charlize Theron - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Charlotte Gainsbourg - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/eng/Charlotte Le Bon - successful deleted!
Excess images in folder :: ./Documents/DataS

...

  0%|          | 0/1277 [00:00<?, ?it/s]

Directory :: ./Documents/DataScience/PET-project/train/women/rus/Агата Вавилова/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/rus/Агата Муцениеце - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Аида Ведищева/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Айса/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/rus/Алёна Ицкова - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Алёна Колесниченко/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Алёна Минчук/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/rus/Алёна Савастова - successful deleted!
Excess images in folder :: ./Documents/DataScience/

Directory :: ./Documents/DataScience/PET-project/train/women/rus/Варвара Маценова/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Венера Рахимова/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Вера Алтайская/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/rus/Вера Брежнева - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Вера Бурлакова/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Вероника Ганич/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/rus/Виктория Агалакова - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Виктория Адельфина/ contains less that 2 images and deleted!
Directory :: ./Documents/D

Directory :: ./Documents/DataScience/PET-project/train/women/rus/Ксения Филиппова/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Лада Сизоненко/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Лариса Буркова/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Лариса Тарковская/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Лейла Киракосян/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Лиана Ваноян/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Лидия Вертинская/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Лидия Волкова/ contains less that 2 images and deleted!
Directory :: ./Docu

Directory :: ./Documents/DataScience/PET-project/train/women/rus/Ольга Власова/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Ольга Глушко/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/rus/Ольга Кабо - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/rus/Ольга Кавалай-Аксенова - successful deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Ольга Кирсанова-Миропольская/ contains less that 2 images and deleted!
Directory :: ./Documents/DataScience/PET-project/train/women/rus/Ольга Кравцова/ contains less that 2 images and deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/rus/Ольга Кузьмина - successful deleted!
Excess images in folder :: ./Documents/DataScience/PET-project/train/women/rus/Ольга Куриленко - successful deleted!
Excess images in folder :: ./Doc

На этом подготовку обучающих датсетов можно считать завершенной и перейти к обучению моделей.